<a href="https://colab.research.google.com/github/vincentjordan27/Named-Entity-Recognition-BILSTM-CRF/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import sys
import csv
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install torchtext==0.6.0

import time
import torch
from torch import nn
from torch.optim import Adam
from torchtext.data import Field, BucketIterator
from torchtext.datasets import SequenceTaggingDataset
from spacy.lang.id import Indonesian

     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 23.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [5]:
maxInt = sys.maxsize

while True:
   
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt / 10)

dataset = '/content/drive/My Drive/Colab Notebooks/SINGGALANG.tsv'
data_df = pd.read_csv(dataset, sep='\t', error_bad_lines=False, header=None, engine="python", names=['word', 'tag'], quoting=csv.QUOTE_NONE)
print(data_df.head())


       word tag
0        Ia   O
1  menjabat   O
2   sebagai   O
3  Presiden   O
4    ketiga   O


In [22]:
class Corpus(object):

  def __init__(self, input, min_word_freq, batch_size):
    
    self.word_field = Field(lower=True)
    self.tag_field= Field(unk_token=None)


In [35]:
corpus = Corpus(
    input=f"{dataset}/",
    min_word_freq=5,
    batch_size=50
    )
#print(corpus.tag_field)